# Exploring Project References

## Instructions


1. Under the **Configuration** section, edit the `slnFolder` to the absolute path of the folder containing your solution.  
2. Under the **Configuration** section, edit the `samplePath` to choose a specific project that will be used by the type provider to examine the project XML. Choose a good sample project with a robust project file that at least includes some project references, or the type provider won't be able to parse your projects.
3. Run all cells in the **Configuration** and **Setting Up** sections.
4. In the **Viewing Project Reference Information** section, run the cells you are interested in.  You may need to edit some project names to view projects you are interested in.  Project names are case-sensitive, so you may want to copy-paste from **A list of all projects under the solution folder**.

The diagram under **All Project References in the solution** gets pretty rough for large projects.  Looking at specific projects will likely be must more useful.

If you switch to a different `slnFolder`, it is best to `Restart` the notebook to make sure that the type provider is refreshed.

## Configuration


In [11]:
[<Literal>]
let slnFolder = @"C:\Dev\Github projects\interactive"

[<Literal>] 
let samplePath = @".\src\Microsoft.Dotnet.Interactive.CSharp\Microsoft.DotNet.Interactive.CSharp.csproj"

let excludeTestProjects = false

## Setting Up


The following cells set up the rest of the notebook.

In [12]:
#r "nuget: FSharp.Data"
#r "nuget: ChickenScratch.Interactive"
open FSharp.Data

Installed Packages ChickenScratch.Interactive, 1.1.0 FSharp.Data, 5.0.2

In [13]:
type ProjectXml = XmlProvider<samplePath, ResolutionFolder=slnFolder>

In [14]:
type PrelimProjectInfo = {
    Name : string
    References : string list
}

module PrelimProjectInfo =
    let getProjectName (str : String) =
        str.Replace(".csproj", "").Replace(".fsproj", "")
    
    
    let FromFileInfo (fi : FileInfo) =
        let proj = ProjectXml.Load fi.FullName
        let refs = proj.ItemGroups |> Seq.collect (fun ig -> 
            ig.ProjectReferences 
            |> Seq.map (fun pr -> pr.Include) 
            |> Seq.filter (fun s -> not (excludeTestProjects && s.Contains "Test")))
            
        let names = refs |> Seq.map (fun r -> FileInfo(r).Name |> getProjectName)
        { Name = fi.Name |> getProjectName ; References = names |> Seq.toList }


    let FromFileInfos fis =
        fis 
        |> Seq.map (fun fi -> FromFileInfo fi)
        |> Seq.filter (fun pi -> not (excludeTestProjects && pi.Name.Contains "Test" )) 
        |> Seq.toList 


    let FromFolder path = 
        DirectoryInfo(slnFolder).GetFiles("*.*proj", SearchOption.AllDirectories)
        |> Seq.filter (fun fi -> fi.Extension <> ".deployproj")
        |> FromFileInfos

In [15]:
type ProjectInfo = {
    Name : string
    References : string list
    ReferencedBy : string list
}


module ProjectInfo =
    let FromPrelim (prelims : PrelimProjectInfo seq) (target : PrelimProjectInfo) =
        {
            Name = target.Name
            References = target.References |> List.sort
            ReferencedBy = 
                prelims
                |> Seq.filter (fun p -> p.References |> Seq.contains target.Name)
                |> Seq.map (fun p -> p.Name)
                |> Seq.sort
                |> Seq.toList
        }

    let FromPrelims (prelims : PrelimProjectInfo seq) =
        prelims |> Seq.map (fun p -> FromPrelim prelims p) |> Seq.sortBy(fun p -> p.Name) |> Seq.toList


    let FromFolder path = path |> PrelimProjectInfo.FromFolder |> FromPrelims


    
        

let projInfos = ProjectInfo.FromFolder slnFolder

In [16]:
type DiagramDirection = 
| LeftToRight
| RightToLeft
| TopToBottom
| BottomToTop
with
    static member toMermaid reverse (dir : DiagramDirection) =
        match dir, reverse with
        | LeftToRight, false 
        | RightToLeft, true -> "direction LR"
        | RightToLeft, false
        | LeftToRight, true -> "direction RL"
        | BottomToTop, true
        | TopToBottom, false-> "direction TB"
        | TopToBottom, true
        | BottomToTop, false -> "direction BT"


module Show =
    open ChickenScratch
    open ChickenScratch.HtmlExpressions

    let getRelationOutput getRelation (extraInfo : string) (infos : ProjectInfo list) proj =
        let rec loop collapsed proj = 
            li { 
                let chkId = Guid.NewGuid().ToString()
                input { _type "checkbox" ; _id chkId ; if collapsed then _checked "checked" }
                label { _for chkId ; proj.Name }
                span { _class "extraInfo" ;  extraInfo }
                ul { 
                    yield! 
                        proj
                        |> getRelation
                        |> List.collect (fun name -> 
                            infos
                            |> List.filter (fun pi -> pi.Name = name) 
                            |> List.map (loop true)
                        )                    
                }
            }            
        
        div {            
            style {
                ".projInfoRefs .extraInfo { font-size: 0.9em; font-style: italic ; color: gray }"
                ".projInfoRefs ul { list-style-type: none; }"
                ".projInfoRefs input { display: none; }"            
                ".projInfoRefs input:checked ~ ul { display: none; }"
                ".projInfoRefs input ~ label { font-weight: bold; }"
                RawContent ".projInfoRefs input ~ label::before { content: '(-) '; }"
                RawContent ".projInfoRefs input:checked ~ label::before { content: '(+) '; }"
            }
            div {
                _class "projInfoRefs"
                ul { loop false (infos |> List.find (fun pi -> pi.Name = proj)) }
            }        
        }
        
    let allNames projInfos =
        div {
            style { 
                ".projInfoNames ul { list-style-type: none; }"
                ".projInfoNames li { font-weight: bold; }"
            }
            div {
                _class "profInfoNames"
                ul {
                    yield! projInfos |> List.map (fun pi -> li { pi.Name })
                }
            }    
        }

    let AllProjectsReferencedBy proj = 
        (getRelationOutput (fun pi -> pi.References) "references:" projInfos proj).Display() |> ignore
    
    let AllProjectsThatReference proj = 
        (getRelationOutput (fun pi -> pi.ReferencedBy) "is referenced by:" projInfos proj).Display() |> ignore

    let AllProjects() = (allNames projInfos).Display() |> ignore

    module Diagram =
        open Microsoft.DotNet.Interactive
        open Microsoft.DotNet.Interactive.Commands
        
        let clean (str : String) = str.Replace(".", "_").Replace("-", "_")

        let getAllRelationOutput getRelation dir (infos: ProjectInfo list) =            
            seq {
                yield "stateDiagram-v2"
                yield dir |> DiagramDirection.toMermaid false
                yield! infos |> Seq.collect (fun pi -> pi |> getRelation |> Seq.map (fun r -> $"{clean pi.Name} --> {clean r}"))
            }
            |> String.concat "\n"

        let rec getRelationOutput getRelation reverse dir (infos: ProjectInfo list) (targets : string list) = 
            let mapToStr target rel =
                if reverse then $"{clean rel} --> {clean target}"
                else $"{clean target} --> {clean rel}"

            let rec loop (targets : string list) = seq {
                match targets with
                | [] -> ()
                | h :: t -> 
                    let target = infos |> List.find (fun pi -> pi.Name = h)
                    let rels = target |> getRelation
                    yield! rels |> Seq.map (mapToStr target.Name)
                    yield! loop (t @ rels |> List.distinct)
            }

            seq {                
                yield "stateDiagram-v2"                
                yield dir |> DiagramDirection.toMermaid reverse
                yield! loop targets
            }
            |> String.concat "\n"

            


        let AllReferences dir = 
            let diagram = getAllRelationOutput (fun pi -> pi.References) dir projInfos            
            Kernel.Root.SendAsync(new SubmitCode(diagram, "mermaid")) |> Async.AwaitTask |> Async.RunSynchronously |> ignore

        let AllProjectsReferencedBy dir targets =
            let diagram = getRelationOutput (fun pi -> pi.References) false dir projInfos targets
            Kernel.Root.SendAsync(new SubmitCode(diagram, "mermaid")) |> Async.AwaitTask |> Async.RunSynchronously |> ignore
            
        let AllProjectsThatReference dir targets =
            let diagram = getRelationOutput (fun pi -> pi.ReferencedBy) true dir projInfos targets
            Kernel.Root.SendAsync(new SubmitCode(diagram, "mermaid")) |> Async.AwaitTask |> Async.RunSynchronously |> ignore        

## Viewing Project Reference Information

### A list of all projects under the solution folder

In [17]:
Show.AllProjects()

BasicConsoleApp 
 ClockExtension 
 FSharpConsole 
 Library 
 Library.InteractiveExtension 
 Library.nuget 
 Microsoft.DotNet.Interactive 
 Microsoft.DotNet.Interactive.ApiCompatibility.Tests 
 Microsoft.DotNet.Interactive.AspNetCore 
 Microsoft.DotNet.Interactive.AspNetCore.Tests 
 Microsoft.DotNet.Interactive.Browser 
 Microsoft.DotNet.Interactive.Browser.Tests 
 Microsoft.DotNet.Interactive.CSharp 
 Microsoft.DotNet.Interactive.CSharp.Tests 
 Microsoft.DotNet.Interactive.CSharpProject 
 Microsoft.DotNet.Interactive.CSharpProject.Tests 
 Microsoft.DotNet.Interactive.Documents 
 Microsoft.DotNet.Interactive.Documents.Tests 
 Microsoft.DotNet.Interactive.ExtensionLab 
 Microsoft.DotNet.Interactive.ExtensionLab.Tests 
 Microsoft.DotNet.Interactive.FSharp 
 Microsoft.DotNet.Interactive.FSharp.Tests 
 Microsoft.DotNet.Interactive.Formatting 
 Microsoft.DotNet.Interactive.Formatting.Tests 
 Microsoft.DotNet.Interactive.Http 
 Microsoft.DotNet.Interactive.HttpRequest 
 Microsoft.DotNet.Interactive.HttpRequest.Tests 
 Microsoft.DotNet.Interactive.Journey 
 Microsoft.DotNet.Interactive.Journey.Tests 
 Microsoft.DotNet.Interactive.Jupyter 
 Microsoft.DotNet.Interactive.Jupyter.Tests 
 Microsoft.DotNet.Interactive.Kql 
 Microsoft.DotNet.Interactive.Kql.Tests 
 Microsoft.DotNet.Interactive.Mermaid 
 Microsoft.DotNet.Interactive.Mermaid.Tests 
 Microsoft.DotNet.Interactive.Netstandard20 
 Microsoft.DotNet.Interactive.PackageManagement 
 Microsoft.DotNet.Interactive.PowerShell 
 Microsoft.DotNet.Interactive.PowerShell.Tests 
 Microsoft.DotNet.Interactive.SqlServer 
 Microsoft.DotNet.Interactive.SqlServer.Tests 
 Microsoft.DotNet.Interactive.Telemetry 
 Microsoft.DotNet.Interactive.Telemetry.Tests 
 Microsoft.DotNet.Interactive.Tests 
 Microsoft.DotNet.Interactive.VSCode 
 Microsoft.Dotnet.Interactive.Shared.shproj 
 RandomNumber 
 SampleExtensions.Tests 
 Tools 
 WpfConnect 
 dotnet-interactive 
 dotnet-interactive.Tests 
 helixpublish.proj 
 interface-generator 
 perf-tests

### All Project References in the solution

In [18]:
Show.Diagram.AllReferences LeftToRight

### References for a specific project

In [19]:
let project = "Microsoft.DotNet.Interactive.FSharp"

Show.AllProjectsReferencedBy project
Show.Diagram.AllProjectsReferencedBy LeftToRight [ project ]

Show.AllProjectsThatReference project
Show.Diagram.AllProjectsThatReference LeftToRight [ project ]


Microsoft.DotNet.Interactive.FSharp 
 references: 
 
 
 
 Microsoft.DotNet.Interactive 
 references: 
 
 
 
 Microsoft.DotNet.Interactive.Documents 
 references: 
 
 
 
 
 Microsoft.DotNet.Interactive.Formatting 
 references: 
 
 
 
 
 
 
 Microsoft.DotNet.Interactive.Formatting 
 references: 
 
 
 
 
 Microsoft.DotNet.Interactive.PackageManagement 
 references: 
 
 
 
 Microsoft.DotNet.Interactive 
 references: 
 
 
 
 Microsoft.DotNet.Interactive.Documents 
 references: 
 
 
 
 
 Microsoft.DotNet.Interactive.Formatting 
 references:

Microsoft.DotNet.Interactive.FSharp 
 is referenced by: 
 
 
 
 Microsoft.DotNet.Interactive.ApiCompatibility.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.ExtensionLab.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.FSharp.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Jupyter 
 is referenced by: 
 
 
 
 Microsoft.DotNet.Interactive.ApiCompatibility.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Jupyter.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Tests 
 is referenced by: 
 
 
 
 Microsoft.DotNet.Interactive.AspNetCore.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Browser.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.CSharp.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.CSharpProject.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Documents.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.HttpRequest.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Journey.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Jupyter.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Mermaid.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.PowerShell.Tests 
 is referenced by: 
 
 
 
 
 dotnet-interactive.Tests 
 is referenced by: 
 
 
 
 
 
 
 dotnet-interactive 
 is referenced by: 
 
 
 
 Microsoft.DotNet.Interactive.AspNetCore.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Tests 
 is referenced by: 
 
 
 
 Microsoft.DotNet.Interactive.AspNetCore.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Browser.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.CSharp.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.CSharpProject.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Documents.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.HttpRequest.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Journey.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Jupyter.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Mermaid.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.PowerShell.Tests 
 is referenced by: 
 
 
 
 
 dotnet-interactive.Tests 
 is referenced by: 
 
 
 
 
 
 
 dotnet-interactive.Tests 
 is referenced by: 
 
 
 
 
 
 
 
 
 Microsoft.DotNet.Interactive.Kql.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.SqlServer.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Tests 
 is referenced by: 
 
 
 
 Microsoft.DotNet.Interactive.AspNetCore.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Browser.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.CSharp.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.CSharpProject.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Documents.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.HttpRequest.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Journey.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Jupyter.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Mermaid.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.PowerShell.Tests 
 is referenced by: 
 
 
 
 
 dotnet-interactive.Tests 
 is referenced by: 
 
 
 
 
 
 
 Microsoft.DotNet.Interactive.VSCode 
 is referenced by: 
 
 
 
 dotnet-interactive 
 is referenced by: 
 
 
 
 Microsoft.DotNet.Interactive.AspNetCore.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Tests 
 is referenced by: 
 
 
 
 Microsoft.DotNet.Interactive.AspNetCore.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Browser.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.CSharp.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.CSharpProject.Tests 
 is referenced by: 
 
 
 


### References for multiple projects

In [20]:
let projects = [ "Microsoft.DotNet.Interactive.ExtensionLab" ; "Microsoft.DotNet.Interactive.Kql" ]

projects |> Seq.iter Show.AllProjectsReferencedBy
Show.Diagram.AllProjectsReferencedBy LeftToRight projects

projects |> Seq.iter Show.AllProjectsThatReference
Show.Diagram.AllProjectsThatReference LeftToRight projects


Microsoft.DotNet.Interactive.ExtensionLab 
 references: 
 
 
 
 Microsoft.DotNet.Interactive 
 references: 
 
 
 
 Microsoft.DotNet.Interactive.Documents 
 references: 
 
 
 
 
 Microsoft.DotNet.Interactive.Formatting 
 references: 
 
 
 
 
 
 
 Microsoft.DotNet.Interactive.CSharp 
 references: 
 
 
 
 Microsoft.DotNet.Interactive 
 references: 
 
 
 
 Microsoft.DotNet.Interactive.Documents 
 references: 
 
 
 
 
 Microsoft.DotNet.Interactive.Formatting 
 references: 
 
 
 
 
 
 
 Microsoft.DotNet.Interactive.Formatting 
 references: 
 
 
 
 
 Microsoft.DotNet.Interactive.PackageManagement 
 references: 
 
 
 
 Microsoft.DotNet.Interactive 
 references: 
 
 
 
 Microsoft.DotNet.Interactive.Documents 
 references: 
 
 
 
 
 Microsoft.DotNet.Interactive.Formatting 
 references: 
 
 
 
 
 
 
 
 
 
 
 Microsoft.DotNet.Interactive.Formatting 
 references:

Microsoft.DotNet.Interactive.Kql 
 references: 
 
 
 
 Microsoft.DotNet.Interactive 
 references: 
 
 
 
 Microsoft.DotNet.Interactive.Documents 
 references: 
 
 
 
 
 Microsoft.DotNet.Interactive.Formatting 
 references: 
 
 
 
 
 
 
 Microsoft.DotNet.Interactive.PackageManagement 
 references: 
 
 
 
 Microsoft.DotNet.Interactive 
 references: 
 
 
 
 Microsoft.DotNet.Interactive.Documents 
 references: 
 
 
 
 
 Microsoft.DotNet.Interactive.Formatting 
 references:

Microsoft.DotNet.Interactive.ExtensionLab 
 is referenced by: 
 
 
 
 Microsoft.DotNet.Interactive.ExtensionLab.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Kql.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.SqlServer.Tests 
 is referenced by:

Microsoft.DotNet.Interactive.Kql 
 is referenced by: 
 
 
 
 Microsoft.DotNet.Interactive.ApiCompatibility.Tests 
 is referenced by: 
 
 
 
 
 Microsoft.DotNet.Interactive.Kql.Tests 
 is referenced by: